In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json


def extract_hotel_info(url):
    # Send a request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract hotel name from the specified h1 tag
        hotel_name_tag = soup.find('h1', class_='x_AX-hotel-name')
        hotel_name = hotel_name_tag.get_text(strip=True) if hotel_name_tag else None
        
        # Extract location from the specified div tag
        location_tag = soup.find('div', class_='x_AX-address')
        location = ', '.join(element.text.strip() for element in location_tag.find_all('span')) if location_tag else None

        # Extract amenities
        amenities_list = soup.select('.t8Xi-amenity-name')
        amenities = ', '.join([amenity.text.strip() for amenity in amenities_list]) if amenities_list else None

        # Extract price
        price_tag = soup.find('span', class_='iWwa-data')
        price = price_tag.text.strip() if price_tag else None
        
        # Extract number of available rooms
        rooms_span = soup.find('span', class_='G2iq-displayText')
        rooms_info = rooms_span['aria-label'] if rooms_span else None

        # Extract check-in and check-out information
        check_in_out_td = soup.find('td', class_='Q6C3-description')
        check_in_out_info = check_in_out_td.text.strip() if check_in_out_td else None

       # Extract customer review information
        review_content_div = soup.find('div', class_='c2oma-review-content')

        if review_content_div:
            review_rating = review_content_div.find('div', class_='c2oma-rating').text.strip() if review_content_div.find('div', class_='c2oma-rating') else None
            review_user_info = review_content_div.find('div', class_='c2oma-user-info').text.strip() if review_content_div.find('div', class_='c2oma-user-info') else None
            review_text_span = review_content_div.find('span', class_='c2oma-review-type-label')

            if review_text_span:
                review_type_label = review_text_span.text.strip()
                review_text = review_text_span.find_next('span').text.strip()
            else:
                review_type_label = None
                review_text = None
        else:
            review_rating = None
            review_user_info = None
            review_type_label = None
            review_text = None

        # Create a DataFrame for hotel information
        data = {
        'Hotel Name': [hotel_name],
        'Location': [location],
        'Amenities': [amenities],
        'Price': [price],
        'Number of Available Rooms': [rooms_info],
        'Other Data': [check_in_out_info],
        'Customer Review Rating': [review_rating],
        'Customer Review User Info': [review_user_info],
        'Customer Review Type Label': [review_type_label],
        'Customer Review Text': [review_text],
        }

        df = pd.DataFrame(data)
        return df
    else:
        print(f'Failed to retrieve data from {url}. Status Code: {response.status_code}')
        return None

def extract_urls_from_json(json_string):
    json_data = json.loads(json_string)
    return [item['url'] for item in json_data['itemListElement']]

def append_new_urls(existing_urls, additional_urls):
    return existing_urls + additional_urls

def run_all_and_display(json_string, existing_urls=None):
    # Extract URLs from JSON
    json_urls = extract_urls_from_json(json_string)

    # If existing_urls is not provided, initialize it as an empty list
    existing_urls = existing_urls or []

    # Append new URLs
    all_urls = append_new_urls(existing_urls, json_urls)

    # List to store DataFrames for each hotel
    hotel_dfs = []

    # Iterate through each URL and call the extract_hotel_info function
    for url in all_urls:
        hotel_df = extract_hotel_info(url)
        if hotel_df is not None:
            hotel_dfs.append(hotel_df)

    # Load the existing DataFrame from the CSV file
    csv_file_path = 'output.csv'
    try:
        existing_df = pd.read_csv(csv_file_path)
    except FileNotFoundError:
        existing_df = pd.DataFrame()

    # Concatenate the list of DataFrames into a single DataFrame
    result_df = pd.concat([existing_df] + hotel_dfs, ignore_index=True)

    # Save the updated DataFrame to the CSV file
    result_df.to_csv(csv_file_path, index=False)
    print(f"DataFrame updated and saved to {csv_file_path}")

    # Automatically append new URLs to existing ones for subsequent runs
    existing_urls.extend(json_urls)

#execution
json_string = '''
{"@context":"http://schema.org","@type":"ItemList","itemListOrder":"http://schema.org/ItemListOrderAscending","description":"16 best hotels in Antipolo","itemListElement":[{"@type":"ListItem","position":1,"url":"https://www.kayak.com.ph/Pasay-Hotels-Urban-Travellers-Hotel.618466.ksp"},{"@type":"ListItem","position":2,"url":"https://www.kayak.com.ph/Paranaque-Hotels-Go-Hotels-Manila-Airport-Road.2677731.ksp"},{"@type":"ListItem","position":3,"url":"https://www.kayak.com.ph/Makati-Hotels-Kl-Serviced-Residences.720304.ksp"},{"@type":"ListItem","position":4,"url":"https://www.kayak.com.ph/Makati-Hotels-Makati-Crown-Regency-Hotel.244296.ksp"},{"@type":"ListItem","position":5,"url":"https://www.kayak.com.ph/Makati-Hotels-Makati-Shangri-La-Hotel-Manila.13762.ksp"},{"@type":"ListItem","position":6,"url":"https://www.kayak.com.ph/Pasay-Hotels-Kabayan-Hotel.319385.ksp"},{"@type":"ListItem","position":7,"url":"https://www.kayak.com.ph/Makati-Hotels-The-Picasso-Boutique-Serviced-Residences.335450.ksp"},{"@type":"ListItem","position":8,"url":"https://www.kayak.com.ph/Makati-Hotels-Guijo-Suites-Makati.639611.ksp"},{"@type":"ListItem","position":9,"url":"https://www.kayak.com.ph/Makati-Hotels-Raffles-Makati.485041.ksp"},{"@type":"ListItem","position":10,"url":"https://www.kayak.com.ph/Makati-Hotels-Valero-Grand-Suites-By-Swiss-Belhotel-Makati.2985932.ksp"},{"@type":"ListItem","position":11,"url":"https://www.kayak.com.ph/Pasay-Hotels-Park-Bed-Breakfast.88383.ksp"},{"@type":"ListItem","position":12,"url":"https://www.kayak.com.ph/Pasay-Hotels-Conrad-Manila.2588129.ksp"},{"@type":"ListItem","position":13,"url":"https://www.kayak.com.ph/Pasay-Hotels-Midas-Hotel-Casino.621294.ksp"},{"@type":"ListItem","position":14,"url":"https://www.kayak.com.ph/Makati-Hotels-Fairmont-Makati.462424.ksp"},{"@type":"ListItem","position":15,"url":"https://www.kayak.com.ph/Pasay-Hotels-Dg-Budget-Hotel-Salem.627120.ksp"},{"@type":"ListItem","position":16,"url":"https://www.kayak.com.ph/Pasay-Hotels-Sofitel-Philippine-Plaza-Manila.324413.ksp"}]}'''

# Call the function to run all and display the result
run_all_and_display(json_string)

DataFrame updated and saved to output.csv
